# Letor Convertor

In [122]:
import pandas as pd
import os
import numpy as np

In [2]:
def getheaders():
    with open('properties.txt', "r") as f:
        properties = f.read().splitlines()
    return properties

In [222]:
def bagging(df):   
    groups = df.groupby('srch_id')
    id_list = np.random.choice(df.srch_id.unique(), df.srch_id.unique().shape[0], replace=True)
    all_indices = []
    for group_id in id_list:
        all_indices += list(groups.get_group(group_id).index.values) 
    return df.iloc[np.array(all_indices)]

In [223]:
def get_target(hotel_set):
    hotel_set['score'] = 0
    hotel_set.loc[hotel_set['click_bool'] == 1, 'score'] = 1
    hotel_set.loc[hotel_set['booking_bool'] == 1, 'score'] = 5
    return hotel_set

In [231]:
def open_set(hotel_set_file, target = True, bagg = False):
    hotel_set = pd.read_csv(hotel_set_file)
    hotel_set.drop(hotel_set.select_dtypes(['object']), inplace=True, axis=1)
    
    if bagging == True:
        hotel_set = bagging(hotel_set).sort_values('srch_id')
        
    if target == True:
        hotel_set = get_target(hotel_set)
        y = hotel_set['score'].values
    else:
        y = None
        
    qid = hotel_set['srch_id'].values
    X = hotel_set[properties].values
        
    return hotel_set, qid, y, X

In [232]:
train_downsampled = '.csv'
train = 'test_files/hoteltest_training_set.csv'
test = 'test_files/hoteltest_testing_set.csv'

In [233]:
properties = getheaders()
#train_set, Tqids, Ty, TX = open_set(train_downsampled, bagging = True)
val_set, Vqids, Vy, VX = open_set(train)
eval_set, Eqids, _, QX = open_set(test, target = False)

# LambdaMART

In [184]:
import pyltr
from sklearn.model_selection import KFold, train_test_split

In [197]:
metric = pyltr.metrics.NDCG(k=5)

In [198]:
model = pyltr.models.LambdaMART(
    metric=metric,
    n_estimators=10,
    learning_rate=0.02,
    max_features='log2',
    query_subsample=0.5,
    max_leaf_nodes=10,
    min_samples_leaf=64,
    verbose=1,
)

In [199]:
#TX, VX, Ty, Vy, Tqids, Vqids = train_test_split(X, y, qids, test_size=0.2, shuffle=False)
print('fitting model...')
#monitor = pyltr.models.monitors.ValidationMonitor(
#        VX, Vy, Vqids, metric=metric)
monitor = None
model.fit(TX, Ty, Tqids, monitor=monitor)

fitting model...
 Iter  Train score  OOB Improve    Remaining                           Monitor Output 
    1       0.0647       0.0317        4.53s                                         
    2       0.1006       0.0370        3.68s                                         
    3       0.1046       0.0088        3.13s                                         
    4       0.1642       0.0095        2.64s                                         
    5       0.1806       0.0105        2.25s                                         
    6       0.2282       0.0492        1.84s                                         
    7       0.2175      -0.0064        1.39s                                         
    8       0.2728      -0.0122        0.92s                                         
    9       0.2673      -0.0010        0.46s                                         
   10       0.2494       0.0014        0.00s                                         


In [200]:
predict = model.predict(TX)
print ('Random ranking:', metric.calc_mean_random(Tqids, Ty))
print ('Our model:', metric.calc_mean(Tqids, Ty, predict))

Random ranking: 0.13131538881499388
Our model: 0.25313775884901657


In [201]:
print(predict.shape)

(19663,)


In [202]:
print(Tqids.shape)
print(TX.shape)

(19663,)
(19663, 29)


In [203]:
result = pd.DataFrame({'srch_id' : Tqids,
                        'prop_id' : train_set.prop_id,
                        'target' : Ty,
                        'score' : predict})

In [204]:
result = result.sort_values(by=['srch_id', 'score'], ascending=[True, False])

In [205]:
result

,srch_id,prop_id,target,score
16,1,88096,0,0.016032
4,1,29604,0,-0.010557
18,1,88218,0,-0.010557
8,1,53341,0,-0.017696
21,1,95307,0,-0.019510
0,1,893,0,-0.020314
6,1,44147,0,-0.028133
14,1,81437,0,-0.028133
5,1,30184,0,-0.030503
9,1,56880,0,-0.030629


In [206]:
result.sort_values(by=['srch_id', 'target'], ascending=[True, False])

,srch_id,prop_id,target,score
12,1,68914,5,-0.096824
16,1,88096,0,0.016032
4,1,29604,0,-0.010557
18,1,88218,0,-0.010557
8,1,53341,0,-0.017696
21,1,95307,0,-0.019510
0,1,893,0,-0.020314
6,1,44147,0,-0.028133
14,1,81437,0,-0.028133
5,1,30184,0,-0.030503
